In [1]:
import numpy as np
import pandas as pd
import requests, json
from datetime import datetime, timedelta
import csv
import time
import re
import psycopg2	
import ETL_pipeline_functions
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from sklearn.neighbors import KernelDensity

In [2]:
# creating a connection to postgresql database
with open('.secret/postgres_credentials.json', 'r') as r:
    postgres_credentials = json.load(r)
    user = postgres_credentials['user']
    password = postgres_credentials['password']

conn = psycopg2.connect(host="localhost", database="running_data", user=user, password=password)

# 1) How has my weekly milage changed over time?

In [82]:
simple_weights = [1/6 for i in range(1,7)] 

linear_weights = [(7-i)/21 for i in range(1,7)] 

exp_factor = np.exp(2/7)
norm_constant = sum([exp_factor ** -i for i in range(1,7)])
exp_weights = [exp_factor ** -i / norm_constant for i in range(1,7)] 

all_weights = [(i, 1/6, (7-i)/21, exp_factor ** -i / norm_constant) for i in range(1,7)] 
values = str(all_weights)[1:-1]

df = pd.read_sql_query("""
WITH sub_1a AS(
SELECT
    date_trunc('week', MIN(timestamp)) AS min_date,
    date_trunc('week', MAX(timestamp)) AS max_date
FROM activities),
sub_1b AS(
SELECT 
    generate_series(min_date, max_date, '7 day'::interval) AS week
FROM sub_1a),
sub_1c AS(
SELECT 
    date_trunc('week', timestamp) AS week,
    SUM(distance) AS total_distance
FROM activities
GROUP BY 1),
week_distances AS(
SELECT
    b.week,
    coalesce(total_distance, 0) AS total_distance
FROM sub_1c c
RIGHT JOIN sub_1b b
ON c.week = b.week),

week_stds AS(
SELECT 
    week,
    total_distance,
    STDDEV(total_distance) OVER(ORDER BY week ROWS BETWEEN 6 PRECEDING AND 1 PRECEDING) AS moving_std
FROM week_distances),

weights (index, simple_weight, linear_weight, exp_weight) AS (VALUES {}),

past_six_weeks_a AS(
SELECT 
    a.week,
    a.total_distance,
    a.moving_std,
    CAST(EXTRACT(EPOCH FROM a.week - b.week) / (3600 * 24 * 7) AS int) AS weeks_before,
    b.total_distance AS before_distance
FROM week_stds a
JOIN week_stds b
ON CAST(EXTRACT(EPOCH FROM a.week - b.week) / (3600 * 24 * 7) AS int) BETWEEN 1 AND 6
ORDER BY 1, 2),

past_six_weeks_b AS(
SELECT 
    week,
    total_distance,
    moving_std,
    before_distance,
    simple_weight,
    linear_weight,
    exp_weight
FROM past_six_weeks_a a
JOIN weights b
ON a.weeks_before = b.index),

moving_averages AS(
SELECT 
    week,
    total_distance,
    moving_std,
    SUM(before_distance * simple_weight) AS simple_moving_avg,
    SUM(before_distance * linear_weight) AS linear_moving_avg,
    SUM(before_distance * exp_weight) AS exp_moving_avg
FROM past_six_weeks_b
WHERE EXTRACT(WEEK FROM week) = 1 OR EXTRACT(YEAR FROM week) > 2018
GROUP BY 1, 2, 3)

SELECT 
    week,
    ROUND(total_distance::numeric, 1) AS total_distance,
    ROUND(simple_moving_avg::numeric, 1) AS moving_avg,
    ROUND((simple_moving_avg - moving_std)::numeric, 1) AS lower_bound,
    ROUND((simple_moving_avg + moving_std)::numeric, 1) AS upper_bound
FROM moving_averages
""".format(values), conn)

In [83]:
# creating figure
fig = go.Figure()

# creating traces
## adding moving average line
fig.add_trace(go.Scatter(x = df.week, y = df.moving_avg, mode='lines', name='6-Week Moving Average', line_color = 'grey', hovertemplate='<b>%{y:}km</b>'))
## adding weekly distance markers
fig.add_trace(go.Scatter(x = df.week, y = df.total_distance, mode='markers', name = 'Week Total', marker = dict(color = 'darkblue'), hovertemplate='<b>%{y:}km</b>'))
## adding upper and lower bounds for moving average line
fig.add_trace(go.Scatter(x = df.week, y = df.upper_bound, fill = None, mode = 'lines', line_color = 'rgba(204, 204, 204, 0)', hoverinfo='skip'))
fig.add_trace(go.Scatter(x = df.week, y = df.lower_bound, fill = 'tonexty', mode = 'lines', line_color = 'rgba(204, 204, 204, 0)', hoverinfo='skip'))

# formatting
## adding annotations for significant events
fig.add_annotation(x=df.week[0],y=df.moving_avg[0],xref="x",yref="y",text="Marathon Training<br>Starts", showarrow=True, arrowhead=0,ax=50,ay=50, font=dict(size = 8))
fig.add_annotation(x=df.week[14],y=df.moving_avg[14],xref="x",yref="y",text="Marathon Week", showarrow=True, arrowhead=0,ax=60,ay=-40, font=dict(size = 8))
fig.add_annotation(x=df.week[41],y=df.moving_avg[41],xref="x",yref="y",text="DS Course<br>Starts", showarrow=True, arrowhead=0,ax=0,ay=-50, font=dict(size = 8))
fig.add_annotation(x=df.week[56],y=df.moving_avg[56],xref="x",yref="y",text="DS Course<br>Ends", showarrow=True, arrowhead=0,ax=0,ay=60, font=dict(size = 8))
fig.add_annotation(x=df.week[64],y=df.moving_avg[64],xref="x",yref="y",text="Lockdown<br>Starts", showarrow=True, arrowhead=0,ax=0,ay=70, font=dict(size = 8))
## formatting axes
fig.update_xaxes(title = 'Date', showgrid = False)
max_y = int(np.ceil(df.total_distance.max() / 10) * 10)
y_ticks = list(range(0,max_y + 16,16))
y_tick_labels = [str(int(y/1.6)) + ' miles' for y in y_ticks]
fig.update_yaxes(title = 'Distance', tickmode = 'array', tickvals = y_ticks, ticktext = y_tick_labels)
## adding title and axis labels
fig.update_layout(title='How has my weekly distance changed over time? <br><sub><sub>A line graph showing the 6-week moving average of my running distance for each week between January 2019 and April 2020.</sub></sub>')
## removing legend
fig.update_layout(showlegend=False)
## changing background theme
fig.update_layout(template = 'plotly_white')
# outputting graph
fig.show()

# 2) How have my running habits changed over time?

In [87]:
values = "('S'), ('M'), ('L'), ('I')"
df = pd.read_sql_query("""
WITH run_types (run_type) AS (VALUES {}),
sub_1a AS(
SELECT
    date_trunc('month', MIN(timestamp)) AS min_date,
    date_trunc('month', MAX(timestamp)) AS max_date
FROM activities),
sub_1b AS(
SELECT 
    generate_series(min_date, max_date, '1 month'::interval) AS month
FROM sub_1a),
sub_1c AS(
SELECT 
    month,
    run_type
FROM sub_1b
CROSS JOIN run_types),
sub_1d AS(
SELECT
    date_trunc('month', timestamp) AS month, 
    run_type, 
    COUNT(*) AS n_runs
FROM activities 
WHERE run_type NOT IN ('WU', 'WD')
GROUP BY 1, 2
ORDER BY 1, 2)
SELECT 
    b.month,
    b.run_type,
    coalesce(n_runs, 0) AS n_runs
FROM sub_1d a
RIGHT JOIN sub_1c b
ON a.month = b.month AND a.run_type = b.run_type
WHERE EXTRACT(YEAR FROM b.month) > 2018;
""".format(values), conn)
df = df.head(len(df) - 4)

In [89]:
df_S = df.loc[df['run_type'] == 'S']
df_M = df.loc[df['run_type'] == 'M']
df_L = df.loc[df['run_type'] == 'L']
df_I = df.loc[df['run_type'] == 'I']

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_S.month, y=df_S.n_runs, mode = 'lines', line = dict(shape = 'spline', width = 15, color = 'rgba(0, 82, 204, 0.5)'), name = 'Short run'))
fig.add_trace(go.Scatter(x=df_M.month, y=df_M.n_runs, mode = 'lines', line = dict(shape = 'spline', width = 15, color = 'rgba(204, 0, 0, 0.5)'), name = 'Mid run'))
fig.add_trace(go.Scatter(x=df_L.month, y=df_L.n_runs, mode = 'lines', line = dict(shape = 'spline', width = 15, color = 'rgba(0, 153, 51, 0.5)'), name = 'Long run'))
fig.add_trace(go.Scatter(x=df_I.month, y=df_I.n_runs, mode = 'lines', line = dict(shape = 'spline', width = 15, color = 'rgba(204, 0, 204, 0.5)'), name = 'Intervals'))

fig.add_trace(go.Scatter(x = list(df_S.month)[::len(list(df_S.month))-1], y = list(df_S.n_runs)[::len(list(df_S.month))-1], mode = 'markers + text', text = ['', list(df_S.n_runs)[-1]], textfont = dict(color = 'white'), marker = dict(size = 25, color = 'rgb(0, 82, 204)'), showlegend = False, hoverinfo = 'skip'))
fig.add_trace(go.Scatter(x = list(df_M.month)[::len(list(df_M.month))-1], y = list(df_M.n_runs)[::len(list(df_M.month))-1], mode = 'markers + text', text = ['', list(df_M.n_runs)[-1]], textfont = dict(color = 'white'), marker = dict(size = 25, color = 'rgb(204, 0, 0)'), showlegend = False, hoverinfo = 'skip'))
fig.add_trace(go.Scatter(x = list(df_L.month)[::len(list(df_L.month))-1], y = list(df_L.n_runs)[::len(list(df_L.month))-1] , mode = 'markers + text', text = ['', list(df_L.n_runs)[-1]], textfont = dict(color = 'white'), marker = dict(size = 25, color = 'rgb(0, 153, 51)'), showlegend = False, hoverinfo = 'skip'))
fig.add_trace(go.Scatter(x = list(df_I.month)[::len(list(df_I.month))-1], y = list(df_I.n_runs)[::len(list(df_I.month))-1], mode = 'markers + text', text = ['', list(df_I.n_runs)[-1]], textfont = dict(color = 'white'), marker = dict(size = 25, color = 'rgb(204, 0, 204)'), showlegend = False, hoverinfo = 'skip'))

fig.update_layout(title = 'How have my running habits changed over time? <br><sub><sub>A bump chart showing how my runs were distributed between run types for each month between January 2019 and April 2020.</sub></sub>')
fig.update_layout(template = 'plotly_white')
fig.update_xaxes(title= 'Date', showgrid=False)
fig.update_yaxes(title = 'Number of runs', showgrid=False)
fig.show()

# 3) How has my running intensity changed over time?

In [94]:
df = pd.read_sql_query("""
WITH zones (zone) AS (VALUES (1), (2), (3), (4), (5)),
weeks AS(
SELECT 
    generate_series(date_trunc('week', MIN(timestamp)), date_trunc('week', MAX(timestamp)), '1 week'::interval) AS week
FROM activities),
weeks_and_zones AS(
SELECT
    week,
    zone
FROM weeks
CROSS JOIN zones),
sub_1 AS(
SELECT 
    date_trunc('week', timestamp) AS week, 
    zone_index AS zone, 
    SUM(b.time) AS time
FROM activities a 
RIGHT JOIN activity_zones b 
ON a.id = b.activity_id 
WHERE zone_type = 'heartrate'
GROUP BY 1, 2),
sub_2 AS (
SELECT 
    b.week,
    b.zone,
    coalesce(time, 0) AS time
FROM sub_1 a
RIGHT JOIN weeks_and_zones b
ON a.week = b.week AND a.zone = b.zone),
sub_3 AS(
SELECT
    week,
    zone,
    time,
    SUM(time) OVER(PARTITION BY zone ORDER BY week) AS moving_sum_zone,    
    SUM(time) OVER(ORDER BY week) AS moving_sum_month
FROM sub_2),
sub_4 AS(
SELECT 
    a.week,
    a.zone,
    a.time,
    a.moving_sum_zone - b.moving_sum_zone AS moving_sum_zone,
    a.moving_sum_month - b.moving_sum_month AS moving_sum_month
FROM sub_3 a
JOIN sub_3 b
ON CAST(EXTRACT(EPOCH FROM a.week - b.week) / (3600 * 24 * 7) AS int) = 6 AND a.zone = b.zone
ORDER BY 1, 2)
SELECT 
    week,
    zone,
    time,
    moving_sum_zone/moving_sum_month * 100 AS moving_percentage
FROM sub_4
WHERE EXTRACT(WEEK FROM week) = 1 OR EXTRACT(YEAR FROM week) > 2018;;
""", conn)

In [95]:
df_1 = df.loc[df['zone'] == 1]
df_2 = df.loc[df['zone'] == 2]
df_3 = df.loc[df['zone'] == 3]
df_4 = df.loc[df['zone'] == 4]
df_5 = df.loc[df['zone'] == 5]

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_1.week, y=df_1.moving_percentage , hoverinfo = 'x+y', mode='lines' , stackgroup = 1, name = 'Zone 1', line_color = 'rgba(255, 230, 230, 0)'))
fig.add_trace(go.Scatter(x=df_2.week, y=df_2.moving_percentage , hoverinfo = 'x+y', mode='lines' , stackgroup = 1, name = 'Zone 2', line_color = 'rgba(255, 153, 153, 0)'))
fig.add_trace(go.Scatter(x=df_3.week, y=df_3.moving_percentage , hoverinfo = 'x+y', mode='lines' , stackgroup = 1, name = 'Zone 3', line_color = 'rgba(255, 77, 77, 0)'))
fig.add_trace(go.Scatter(x=df_4.week, y=df_4.moving_percentage , hoverinfo = 'x+y', mode='lines' , stackgroup = 1, name = 'Zone 4', line_color = 'rgba(255, 0, 0, 0)'))
fig.add_trace(go.Scatter(x=df_5.week, y=df_5.moving_percentage , hoverinfo = 'x+y', mode='lines' , stackgroup = 1, name = 'Zone 5', line_color = 'rgba(179, 0, 0, 0)'))

fig.update_layout(title = 'How has the intensity of my training changing over time? <br><sub><sub>A stacked area chart showing how my time was distributed between heart rate zones for each moving 6-week period between January 2019 and April 2020.</sub></sub>')
fig.update_xaxes(title= 'Date', showgrid = False)
fig.update_yaxes(title = 'Percentage of Time', range = (0, 100), tickvals = list(range(0, 120, 20)), ticktext = [str(x) + '%' for x in range(0,120,20)], showgrid = False)
fig.show()

# 4) How are my runs distributed thoughout the week?

In [92]:
df = pd.read_sql_query("""
WITH run_types (run_type) AS (VALUES ('S'), ('M'), ('L'), ('I')),
days (day) AS (VALUES (1), (2), (3), (4), (5), (6), (7)),
days_and_types AS(
SELECT
    day,
    run_type
FROM days
CROSS JOIN run_types),
sub_1 AS(
SELECT 
    EXTRACT(ISODOW FROM timestamp) AS day,
    run_type,
    coalesce(COUNT(*), 0) AS n_runs
FROM activities
WHERE run_type NOT IN ('WU', 'WD') AND EXTRACT(YEAR FROM timestamp) > 2018
GROUP BY 1, 2
ORDER BY 1, 2)
SELECT 
    b.day,
    b.run_type,
    coalesce(a.n_runs, 0) AS n_runs
FROM sub_1 a
RIGHT JOIN days_and_types b
ON a.day = b.day AND a.run_type = b.run_type
ORDER BY 1, 2;
""", conn)
df_pivot = df.pivot(index='run_type', columns='day', values='n_runs')
z = [list(run_type) for run_type in np.array(df_pivot)]

In [93]:
fig = ff.create_annotated_heatmap(z, x = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], y = ['Intervals', 'Long run', 'Mid run', 'Short run'], annotation_text = z, colorscale = 'Greys', hoverinfo = 'skip')
fig.update_layout(title_text='How are my runs distributed throughout the week? <br><sub><sub>A heatmap showing how my runs were distributed between run types for each day of the week during the period between January 2019 and April 2020.')
fig.update_layout(xaxis = dict(side="bottom"))
fig.show()